In [4]:
from glob import glob
from tqdm import tqdm

files = glob('embedding-pair-mining/*.jsonl')
files = [f for f in files if 'test' not in f]
files

['embedding-pair-mining/malay-news-dataset-bge-train.jsonl',
 'embedding-pair-mining/twitter-dataset.jsonl',
 'embedding-pair-mining/carigold-dataset.jsonl',
 'embedding-pair-mining/hansard-dataset-bge.jsonl',
 'embedding-pair-mining/facebook-dataset.jsonl',
 'embedding-pair-mining/iium-dataset-bge.jsonl',
 'embedding-pair-mining/twitter-dataset-bge-train.jsonl',
 'embedding-pair-mining/b.cari.com.my-dataset-bge-train.jsonl',
 'embedding-pair-mining/b.cari.com.my-dataset.jsonl',
 'embedding-pair-mining/facebook-dataset-bge.jsonl',
 'embedding-pair-mining/manglish-dataset-bge.jsonl',
 'embedding-pair-mining/lowyat-dataset.jsonl',
 'embedding-pair-mining/c.cari.com.my-dataset-bge-train.jsonl']

In [6]:
with open('train-embedding.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            for l in tqdm(fopen):
                fopen_l.write(l)

3920774it [00:17, 227429.96it/s]
8706it [00:00, 239404.24it/s]
436947it [00:03, 119153.22it/s]
275460it [01:01, 4466.21it/s] 
103034it [00:05, 17688.68it/s]
1001079it [00:40, 24636.63it/s]
2062574it [00:16, 128669.33it/s]
1885628it [01:34, 20030.55it/s]
19712it [00:07, 2788.62it/s]
425553it [00:01, 355727.29it/s]
3108939it [00:14, 220081.72it/s]
185318it [00:25, 7170.20it/s] 
1410413it [02:32, 9236.93it/s] 


In [16]:
import datasets

dataset = datasets.load_dataset('json', data_files='train-embedding.jsonl', split='train',
                               cache_dir = 'train-embedding')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/ubuntu/train-embedding/json/default-cf7f96519654c781/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.
